In [1]:
#import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import ftplib
from matplotlib.dates import DateFormatter
import ipywidgets as widgets
%matplotlib widget


In [38]:
#ftp server data
hostname = "ftpupload.net"
username = "epiz_32799455"
pwd = "2RerU5lOOo5s9"
folder = 'shared_files'
filename = 'flight_fares.csv'



#download the file
downloaded_filename = 'downl_'+filename
with ftplib.FTP(hostname, username, pwd) as ftp:
    try:
        ftp.cwd(folder)
        ftp.encoding = "utf-8"
        with open(downloaded_filename, 'wb+') as f:
            ftp.retrbinary("RETR " + filename ,f.write)
            f.seek(0)
            new_string = f.read().decode("utf-8") 
            
    except:
        print ("Error")

#print(new_string)

Query date	Label	ID	Origin	Destination	Date	Price	Flight number	Flight string	Price Updated
2022-10-08 01:28:40	Navidad	1	PSA	TFS	2022-12-29T11:45:00	260.79	FR7476	FR~7476~ ~~PSA~12/29/2022 11:45~TFS~12/29/2022 15:15~~	1665150109000
2022-10-08 01:28:40	Navidad	2	PSA	TFS	2022-12-31T06:55:00	229.19	FR7476	FR~7476~ ~~PSA~12/31/2022 06:55~TFS~12/31/2022 10:25~~	1665162458000
2022-10-08 01:28:40	Navidad	3	TFS	PSA	2023-01-05T15:50:00	229.19	FR7477	FR~7477~ ~~TFS~01/05/2023 15:50~PSA~01/05/2023 21:00~~	1665127313000
2022-10-08 01:28:40	Navidad	4	TFS	PSA	2023-01-07T11:00:00	354.42	FR7477	FR~7477~ ~~TFS~01/07/2023 11:00~PSA~01/07/2023 16:10~~	1665162460000
2022-10-08 01:28:40	Navidad	5	TFS	PSA	2023-01-09T10:15:00	229.19	FR7477	FR~7477~ ~~TFS~01/09/2023 10:15~PSA~01/09/2023 15:25~~	1665156559000
2022-10-09 14:41:37	Navidad	1	PSA	TFS	2022-12-29T11:45:00	260.79	FR7476	FR~7476~ ~~PSA~12/29/2022 11:45~TFS~12/29/2022 15:15~~	1665300187000
2022-10-09 14:41:37	Navidad	2	PSA	TFS	2022-12-31T06:55:00	229.

In [55]:
df_all = pd.read_csv(downloaded_filename, sep = '\t',parse_dates=['Query date'],dtype = {'Price': 'float64'}, na_values={'Price': 'Err'})
df_all = df_all.dropna()
df_list_label = [x for _, x in df_all.groupby(df_all['Label'])]
label_list = [df.iloc[0]['Label'] for df in df_list_label]
df_list_ID = [[x for _, x in df.groupby(df['ID'])] for df in df_list_label] 


    

### Ryan air monitoring app
Indicate the labels to display to plot the price evolution.
You can toggle the zoom on the lateral sidebar, and you can toggle the visibility of the lines by clicking on the legend lines.

In [54]:


check_list = [widgets.Checkbox(value=False, description=lab, indent=False) for lab in label_list]
check_list[-1].value = True

display(widgets.GridBox(check_list, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)")))

fig,ax = plt.subplots(figsize = [8,4])


def onpick(event):
    # on the pick event, find the orig line corresponding to the
    # legend proxy line, and toggle the visibility
    legline = event.artist
    origline = fig.line_dict[legline]
    vis = not origline.get_visible()
    origline.set_visible(vis)
    # Change the alpha on the line in the legend so we can see what lines
    # have been toggled
    if vis:
        legline.set_alpha(1.0)
        legline._legmarker.set_alpha(1)
    else:
        legline.set_alpha(0.2)
        legline._legmarker.set_alpha(0.2)
    fig.canvas.draw()



def plot_sel(change):
    ax.clear()
    line_list = []
    for ii, df_list in enumerate(df_list_ID):
        if check_list[ii].value:
            for df in df_list:
                tmp=df.iloc[0]
                flight_descript = f"{tmp['Label']} {tmp['Origin']}-{tmp['Destination']} {tmp['Date']}"
                if tmp['Origin'] == 'PSA' or tmp['Origin'] == 'FLR':
                    symbol = '.-'
                else:
                    symbol = '^-'
                line_list += ax.plot(df['Query date'],df['Price'],symbol,label=flight_descript)
    leg=ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    leg.get_frame().set_alpha(0.4)
    
    fig.line_dict = dict()
    for legline, origline in zip(leg.get_lines(), line_list):
        #legline.set_picker(5)  # 5 pts tolerance
        legline.set_picker(True)
        legline.set_pickradius(5)  # 5 pts tolerance
        fig.line_dict[legline] = origline
    
    ax.grid('True')
    ax.set_xlabel('Query date')
    ax.set_ylabel('Price (€)')
    ax.xaxis.set_major_formatter(DateFormatter('%d/%m'))
    fig.tight_layout()
    fig.canvas.mpl_connect('pick_event', onpick)

    


plot_sel(None)
[cl.observe(plot_sel) for cl in check_list]

None

# link_to_view_edit = 'https://docs.google.com/spreadsheets/d/1U3UlqFHcbvpml8QwxAMaEMRqDGM2bqYO?rtpof=true&authuser=claudiooton%40gmail.com&usp=drive_fs'
# #link = widgets.HTML('Link to change flight monitor table:')
# link = widgets.HTML(value="<a href=""https://docs.google.com/spreadsheets/d/1U3UlqFHcbvpml8QwxAMaEMRqDGM2bqYO?rtpof=true&authuser=claudiooton%40gmail.com&usp=drive_fs"" target=""_blank"">Click here to edit flight monitor table</a>")

# #display(text)
# display(link)


GridBox(children=(Checkbox(value=False, description='Navidad', indent=False), Checkbox(value=False, descriptio…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<a href="https://docs.google.com/spreadsheets/d/1U3UlqFHcbvpml8QwxAMaEMRqDGM2bqYO?rtpof=true&authuser=claudiooton%40gmail.com&usp=drive_fs" target="_blank">Click here to edit flight monitor table</a>